In [1]:
#imports
import pandas as pd
from fastai.tabular.all import *

In [2]:
tracks = pd.read_csv('tracks.csv')
tracks.head()

,genre,band_name,album_name,album_id,track_title,track_id,preview_downloaded
0,thrash metal,Metallica,72 Seasons,6UwjRSX9RQyNgJ3LwYhr9i,72 Seasons,3uSATa7xWXVJrMhJJSnGGk,N
1,thrash metal,Metallica,72 Seasons,6UwjRSX9RQyNgJ3LwYhr9i,Shadows Follow,5iG1GycXpheJ5xMW0gNvKv,N
2,thrash metal,Metallica,72 Seasons,6UwjRSX9RQyNgJ3LwYhr9i,Screaming Suicide,2hBn08QphNtdnhwDNtnZMg,N
3,thrash metal,Metallica,72 Seasons,6UwjRSX9RQyNgJ3LwYhr9i,Sleepwalk My Life Away,6HeYgpgqEo8SGis4dHwRbu,N
4,thrash metal,Metallica,72 Seasons,6UwjRSX9RQyNgJ3LwYhr9i,You Must Burn!,1ALXFmNr7lrTug2LAE989q,N


In [3]:
features = pd.read_csv('audio_features.csv')
features.head()

,id,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness_dB,mode,speechiness,tempo,time_signature,valence
0,3uSATa7xWXVJrMhJJSnGGk,0.000042,0.196,459120,0.981,0.0520,9,0.0636,-3.865,1,0.0574,168.183,4,0.403
1,5iG1GycXpheJ5xMW0gNvKv,0.000106,0.316,371880,0.978,0.0325,11,0.3250,-3.937,1,0.0507,178.554,4,0.671
2,2hBn08QphNtdnhwDNtnZMg,0.000027,0.427,330453,0.973,0.0200,4,0.2600,-3.469,0,0.0445,100.979,4,0.317
3,6HeYgpgqEo8SGis4dHwRbu,0.000021,0.450,416067,0.949,0.0404,11,0.0977,-4.181,1,0.0343,124.949,4,0.530
4,1ALXFmNr7lrTug2LAE989q,0.000090,0.235,423013,0.958,0.0173,4,0.0374,-3.069,1,0.0426,172.774,4,0.453


In [4]:
joined = features.merge(tracks, how='left', left_on='id', right_on='track_id')
print(joined.shape)
joined.columns    

(276792, 21)


Index(['id', 'acousticness', 'danceability', 'duration_ms', 'energy',
       'instrumentalness', 'key', 'liveness', 'loudness_dB', 'mode',
       'speechiness', 'tempo', 'time_signature', 'valence', 'genre',
       'band_name', 'album_name', 'album_id', 'track_title', 'track_id',
       'preview_downloaded'],
      dtype='object')

In [5]:
data = joined[['track_id', 'acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 'key', 'liveness', 'loudness_dB', 'mode', 'speechiness', 'tempo', 'time_signature', 'valence', 'genre']]
data.columns

Index(['track_id', 'acousticness', 'danceability', 'duration_ms', 'energy',
       'instrumentalness', 'key', 'liveness', 'loudness_dB', 'mode',
       'speechiness', 'tempo', 'time_signature', 'valence', 'genre'],
      dtype='object')

In [12]:
dls = TabularDataLoaders.from_df(data, path='.', y_names='genre', procs=[Categorify, FillMissing, Normalize],
                                 cont_names=['acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 'key', 
                                             'liveness', 'loudness_dB', 'speechiness', 'tempo', 'time_signature', 'valence'],
                                 cat_names=['mode'])

In [13]:
learn = tabular_learner(dls, metrics=accuracy)

In [15]:
learn.fit_one_cycle(30)

epoch,train_loss,valid_loss,accuracy,time
0,2.187003,2.191720,0.320586,00:27
1,2.208564,2.188648,0.320062,00:43
2,2.167512,2.186973,0.321128,01:10
3,2.207386,2.194678,0.319087,01:09
4,2.181768,2.183844,0.323639,01:05
5,2.234936,2.186712,0.320983,01:05
6,2.218942,2.184821,0.321543,01:08
7,2.184012,2.182875,0.324163,01:06
8,2.167479,2.170767,0.326150,01:07
9,2.181605,2.173374,0.324380,01:08


c:\Users\MKnaepen\Documents\BeCode\projects\metal classifier\venv\Lib\site-packages\fastai\learner.py:448: SyntaxWarning: invalid escape sequence '\R'
  e.args = [f"Custom classes or functions exported with your `Learner` not available in namespace.\Re-declare/import before loading:\n\t{e.args[0]}"]


KeyboardInterrupt: 

In [7]:
# key and time sig to categorical, no normalization
dls = TabularDataLoaders.from_df(data, path='.', y_names='genre', procs=[Categorify, FillMissing],
                                 cont_names=['acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 
                                             'liveness', 'loudness_dB', 'speechiness', 'tempo', 'valence'],
                                 cat_names=['mode', 'key', 'time_signature'])
learn = tabular_learner(dls, metrics=accuracy)
learn.fit_one_cycle(10)

epoch,train_loss,valid_loss,accuracy,time
0,2.267125,2.259373,0.300715,00:27
1,2.253346,2.220812,0.309242,00:28
2,2.211592,2.190276,0.317732,00:28
3,2.208149,2.179838,0.323278,00:28
4,2.201422,2.171094,0.324307,00:27
5,2.151778,2.161160,0.327324,00:28
6,2.156180,2.149885,0.330431,00:28
7,2.162551,2.145735,0.331786,00:33
8,2.145067,2.143544,0.332418,01:04
9,2.129818,2.141339,0.333105,00:58


In [9]:
# normalize the data again
dls = TabularDataLoaders.from_df(data, path='.', y_names='genre', procs=[Categorify, FillMissing, Normalize],
                                 cont_names=['acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 
                                             'liveness', 'loudness_dB', 'speechiness', 'tempo', 'valence'],
                                 cat_names=['mode', 'key', 'time_signature'])
learn = tabular_learner(dls, metrics=accuracy)
learn.fit_one_cycle(10)

epoch,train_loss,valid_loss,accuracy,time
0,2.282353,2.260648,0.299902,00:30
1,2.247715,2.224260,0.310163,00:30
2,2.204045,2.201272,0.314914,00:26
3,2.174590,2.187334,0.321399,00:54
4,2.199508,2.186581,0.321074,01:00
5,2.192861,2.166346,0.327631,00:58
6,2.152860,2.156623,0.331732,00:58
7,2.158260,2.184231,0.318183,01:04
8,2.145039,2.151820,0.332888,00:56
9,2.138298,2.151962,0.332725,00:51
